In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import binascii
import json
from pathlib import Path
from datetime import datetime
from sqlalchemy import create_engine
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import pkcs7
from typing import Tuple, Optional, List

# matplotlib の設定
import matplotlib
matplotlib.rc('font', family='Noto Sans CJK JP')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

# データベース設定ファイルのパス
config_path = Path('/home/asomura/waseda/nextstep/RAPIDS/config/database.json')

def get_engine(db_name: str) -> create_engine:
    """指定したデータベースのエンジンを作成"""
    host = 'localhost'
    with open(config_path) as f:
        config = json.load(f)['database']
    return create_engine(
        f'postgresql://{config["user"]}:{config["password"]}@{host}/{db_name}'
    )

def analyze_registrar_distribution(db_name: str) -> pd.DataFrame:
    """
    指定したデータベースのレジストラ分布を解析

    Args:
        db_name: データベース名

    Returns:
        レジストラ分布を含む DataFrame
    """
    engine = get_engine(db_name)
    query = """
    SELECT 
        domain_registrar,
        COUNT(*) as count,
        CAST((COUNT(*)::float * 100 / SUM(COUNT(*)) OVER()) as numeric(10,2)) as percentage
    FROM website_data
    WHERE status = 7 
        AND domain_registrar IS NOT NULL 
        AND domain_registrar != ''
    GROUP BY domain_registrar
    ORDER BY count DESC
    LIMIT 20
    """
    return pd.read_sql_query(query, engine)

def plot_registrar_distribution(df: pd.DataFrame, db_name: str, output_dir: Path) -> None:
    """
    レジストラ分布の可視化を作成し、保存

    Args:
        df: レジストラデータを含む DataFrame
        db_name: データベース名
        output_dir: プロットの保存先ディレクトリ
    """
    plt.figure(figsize=(15, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, 10))
    plt.barh(range(10), df.head(10)['percentage'], color=colors)
    plt.yticks(range(10), df.head(10)['domain_registrar'])
    
    plt.title(f'Top 10 Registrars Distribution - {db_name}')
    plt.xlabel('Percentage (%)')
    plt.ylabel('Registrar')
    
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = output_dir / f'registrar_distribution_{db_name}_{timestamp}.png'
    plt.savefig(output_path, bbox_inches='tight', dpi=300)
    plt.close()

def save_results(df: pd.DataFrame, db_name: str, output_dir: Path) -> None:
    """
    解析結果を CSV で保存

    Args:
        df: 解析結果を含む DataFrame
        db_name: データベース名
        output_dir: 結果の保存先ディレクトリ
    """
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_path = output_dir / f'registrar_distribution_{db_name}_{timestamp}.csv'
    df.to_csv(output_path, index=False)
    print(f"[INFO] Registrar results saved to: {output_path}")

# ----- 以下、証明書データ解析用の追加処理 -----

def clean_cert_data(cert_data: str) -> Optional[bytes]:
    """証明書データのクリーニングとバイナリ変換"""
    try:
        cert_data = cert_data.strip()
        if "-----BEGIN" in cert_data:
            return cert_data.encode('utf-8')
        hex_str = re.sub(r'[^0-9a-fA-F]', '', cert_data.replace('\\x', '').replace(' ', ''))
        if len(hex_str) % 2 != 0:
            hex_str += '0'
        return binascii.unhexlify(hex_str)
    except Exception as e:
        print(f"[ERROR] 証明書データ変換失敗: {str(e)}")
        return None

def load_certificate_from_data(data: bytes) -> Tuple[Optional[x509.Certificate], str]:
    """
    PEM, DER, PKCS#7（PEM/DER）の各形式に対応して証明書をロード

    Returns:
        (証明書オブジェクトまたは None, 形式を示す文字列)
    """
    try:
        if b'-----BEGIN CERTIFICATE-----' in data:
            return x509.load_pem_x509_certificate(data, default_backend()), "PEM"
        if b'-----BEGIN PKCS7-----' in data:
            return pkcs7.load_pem_pkcs7_certificates(data)[0], "PKCS7_PEM"
        try:
            return x509.load_der_x509_certificate(data, default_backend()), "DER"
        except Exception:
            return pkcs7.load_der_pkcs7_certificates(data)[0], "PKCS7_DER"
    except Exception as e:
        print(f"[ERROR] 証明書のロード失敗: {str(e)}")
        return None, "Unknown"

def analyze_certificates_from_db(db_name: str) -> Tuple[pd.DataFrame, int]:
    """
    指定したデータベース内の証明書データを解析し、パース可能な証明書の数を確認する

    ※証明書データは website_data テーブルの https_certificate_body カラムから取得します。

    Returns:
        (証明書情報をまとめた DataFrame, DBから取得した総レコード数)
    """
    engine = get_engine(db_name)
    query = """
    SELECT id, https_certificate_body FROM website_data
    WHERE status = 7 AND https_certificate_body IS NOT NULL AND https_certificate_body != ''
    """
    df_db = pd.read_sql_query(query, engine)
    total_records = len(df_db)
    records: List[dict] = []
    
    for _, row in df_db.iterrows():
        cert_data_raw = row['https_certificate_body']
        data = clean_cert_data(cert_data_raw)
        if not data:
            print(f"[ERROR] 証明書データ変換失敗: レコードID {row['id']}")
            continue
        cert, cert_format = load_certificate_from_data(data)
        if cert:
            # もし1レコードに複数の証明書が含まれている場合、ここでループ処理に変更することも可能です
            records.append({
                "record_id": row['id'],
                "cert_format": cert_format,
                "subject": cert.subject.rfc4514_string(),
                "issuer": cert.issuer.rfc4514_string(),
                "not_before": cert.not_valid_before,
                "not_after": cert.not_valid_after
            })
        else:
            print(f"[ERROR] 証明書パース失敗: レコードID {row['id']}, フォーマット: {cert_format}")
    
    df_cert = pd.DataFrame(records)
    processed_cert_count = len(df_cert)
    print(f"[INFO] {db_name} - DB総レコード数: {total_records}, CSV出力件数: {processed_cert_count}")
    return df_cert, total_records

def check_certificate_count(csv_cert_count: int, db_record_count: int, db_name: str) -> None:
    """
    CSVに書かれた証明書の数と、DBのレコード数を比較し、違いがある場合は理由を出力する
    """
    if csv_cert_count == db_record_count:
        print(f"[INFO] {db_name}: CSVに書かれた証明書の数 ({csv_cert_count}) とDBのレコード数 ({db_record_count}) は一致しています。")
    else:
        print(f"[ERROR] {db_name}: CSVに書かれた証明書の数 ({csv_cert_count}) とDBのレコード数 ({db_record_count}) が一致しません。")
        if csv_cert_count < db_record_count:
            print("→ 一部のレコードで証明書の変換またはパースに失敗している可能性があります。")
        elif csv_cert_count > db_record_count:
            print("→ 一部のレコードに複数の証明書が含まれている可能性があります。")

# ----- メイン処理 -----

output_dir = Path('/home/asomura/waseda/nextstep/RAPIDS/reports/database_analysis')
output_dir.mkdir(parents=True, exist_ok=True)

# 既存のレジストラ分布分析
results = {}
for db_name in ['website_data', 'normal_sites']:
    print(f"\nAnalyzing registrar distribution for {db_name}...")
    try:
        df_registrar = analyze_registrar_distribution(db_name)
        results[db_name] = df_registrar

        print(f"\nTop 10 Registrars for {db_name}:")
        print(df_registrar.head(10))
        
        plot_registrar_distribution(df_registrar, db_name, output_dir)
        save_results(df_registrar, db_name, output_dir)
        
        print(f"[INFO] Registrar analysis completed for {db_name}")
    except Exception as e:
        print(f"[ERROR] Error analyzing registrar distribution for {db_name}: {str(e)}")

# 証明書解析とCSV出力、件数チェック
for db_name in ['website_data', 'normal_sites']:
    print(f"\nAnalyzing certificates from database: {db_name}...")
    try:
        df_cert, total_records = analyze_certificates_from_db(db_name)
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        cert_output_path = output_dir / f'certificate_analysis_{db_name}_{timestamp}.csv'
        df_cert.to_csv(cert_output_path, index=False)
        print(f"[INFO] Certificate analysis results saved to: {cert_output_path}")
        
        csv_cert_count = len(df_cert)
        check_certificate_count(csv_cert_count, total_records, db_name)
    except Exception as e:
        print(f"[ERROR] Error analyzing certificates for {db_name}: {str(e)}")

# registrar_distributionの比較分析とCSV生成
if len(results) == 2:
    print("\nComparing registrar distributions between website_data and normal_sites...")
    comparison_df = pd.merge(
        results['website_data'].rename(columns={'percentage': 'website_data_percentage'}),
        results['normal_sites'].rename(columns={'percentage': 'normal_sites_percentage'}),
        on='domain_registrar',
        how='outer'
    ).fillna(0)
    comparison_df = comparison_df.sort_values('website_data_percentage', ascending=False)
    
    print("\nTop 10 registrars (comparison):")
    print(comparison_df[['domain_registrar', 'website_data_percentage', 'normal_sites_percentage']].head(10))
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    comparison_path = output_dir / f'registrar_comparison_{timestamp}.csv'
    comparison_df.to_csv(comparison_path, index=False)
    print(f"[INFO] Registrar comparison results saved to: {comparison_path}")

# ※いったん読み込み、次の指示を待ちます。



Analyzing registrar distribution for website_data...

Top 10 Registrars for website_data:
      domain_registrar  count  percentage
0            Gandi SAS   3376       29.29
1     MarkMonitor Inc.   1410       12.23
2        阿里云计算有限公司（万网）   1299       11.27
3     GoDaddy.com, LLC    835        7.24
4  Gname.com Pte. Ltd.    722        6.26
5     Cloudflare, Inc.    343        2.98
6    MarkMonitor, Inc.    330        2.86
7   Tucows Domains Inc    209        1.81
8         Dynadot Inc.    143        1.24
9         TUCOWS, INC.    136        1.18
[INFO] Registrar results saved to: /home/asomura/waseda/nextstep/RAPIDS/reports/database_analysis/registrar_distribution_website_data_20250316_195631.csv
[INFO] Registrar analysis completed for website_data

Analyzing registrar distribution for normal_sites...

Top 10 Registrars for normal_sites:
                                   domain_registrar  count  percentage
0                                  GoDaddy.com, LLC   1219       16.05
1      

/tmp/ipykernel_174867/1247351425.py:166: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_before_utc.
  "not_before": cert.not_valid_before,
/tmp/ipykernel_174867/1247351425.py:167: CryptographyDeprecationWarning: Properties that return a naïve datetime object have been deprecated. Please switch to not_valid_after_utc.
  "not_after": cert.not_valid_after


[INFO] website_data - DB総レコード数: 12074, CSV出力件数: 12074
[INFO] Certificate analysis results saved to: /home/asomura/waseda/nextstep/RAPIDS/reports/database_analysis/certificate_analysis_website_data_20250316_195633.csv
[INFO] website_data: CSVに書かれた証明書の数 (12074) とDBのレコード数 (12074) は一致しています。

Analyzing certificates from database: normal_sites...
[INFO] normal_sites - DB総レコード数: 9591, CSV出力件数: 9591
[INFO] Certificate analysis results saved to: /home/asomura/waseda/nextstep/RAPIDS/reports/database_analysis/certificate_analysis_normal_sites_20250316_195634.csv
[INFO] normal_sites: CSVに書かれた証明書の数 (9591) とDBのレコード数 (9591) は一致しています。

Comparing registrar distributions between website_data and normal_sites...

Top 10 registrars (comparison):
       domain_registrar  website_data_percentage  normal_sites_percentage
12            Gandi SAS                    29.29                     1.78
19     MarkMonitor Inc.                    12.23                     1.37
31        阿里云计算有限公司（万网）                   